# Fonctions

Attention on doit distinguer plusieurs cas dans nos calculs : 
- Barre, on ne transmet que des efforts axiaux
- Poutre Euler-Bernoulli, on ne transmet que des efforts transversaux et moment de flexion
- Poutre Complete, on transmet les efforts axiaux, transversaux et moment

## 1. Fonctions modèle barre

In [3]:
def bc_barre(NoN) :
    Nb_ddl = NoN
    BC = np.eye(Nb_ddl)
    k = int(input("Combien de ddl a bloquer ?"))
    for i in range(k):
        elem = int(input("element concerne ?"))
        BC = np.delete(BC,[elem-1],axis=1)
    return BC
    
#TODO gérer si input est vide pour u v et phi
def apply_forces_barre(NoN) : 
    k = int(input("Combien de forces a appliquer ?"))
    F = np.zeros([NoN*2,1])
    for i in range(k):
        elem = int(input("element concerne ? "))
        valeur = int(input("Norme de la force : "))
        F[(elem-1)*2] = valeur
    return F

def assemblage(M_elem,BB,NL,EL) :
    M_global = np.zeros([len(NL),len(NL)])
    for i in range(len(EL)) :
        M_global = M_global + changement_base(BB[i],M_elem)
    return M_global

def changement_base(P,M) : 
    return P.dot(M).dot(np.transpose(P))

def changement_coord(NL,EL) :
    BB = []
    for i in range(len(EL)) : 
        B = np.zeros([len(NL),2])
        noeud1 = EL[i,0]
        noeud2 = EL[i,1]
        B[(noeud1 - 1), 0] = 1
        B[(noeud2 - 1), 1] = 1
        BB.append(B)
    return BB

In [ ]:
def solve_barre(NL,EL,F,Q_equi,BC,affiche=0) :
    ### On calcule la matrice de raideur dans le repere local dans les coordonnees locales
    K_elem = E*S/L*np.array([[1, -1],
                            [-1, 1]])
    
    ### Assemblage de la matrice globale 
    matrices_de_passage = changement_coord(NL,EL)
    K_glob = assemblage(K_elem,matrices_de_passage,NL,EL)
    K_glob_r = np.transpose(BC).dot(K_glob).dot(BC)
    F_r = np.transpose(BC).dot(F.flatten() + Q_equi)
    ### Calcul des deplacements et reactions
    U_r = inv(K_glob_r).dot(F_r)
    U = BC.dot(U_r)
    R = K_glob.dot(U) - F.flatten() - Q_equi
    if affiche == 1 :
        print("Matrice de condition limite :\n",BC)
        print("Forces appliquees :\n",F)
        print("Matrice de raideur globale :\n",K_glob)
        print("Deplacements :\n",U)
        print("Reactions :\n",R)
    return U, R

def geom(NL) : 
    x = [x for x in NL[:]]
    y = [0 for x in NL[:]]
    size = 200
    offset = size/40000.
    plt.scatter(x, y, c='y', s=size, zorder=5)
    for i, location in enumerate(zip(x,y)):
        plt.annotate(i+1, (location[0]-offset, location[1]-offset), zorder=10)
    for i in range(len(EL)) :
        xi,xj = NL[EL[i,0]-1,0],NL[EL[i,1]-1,0]
        yi,yj = 0,0
        plt.plot([xi,xj],[yi,yj],color = 'k', lw = 1, linestyle = '--')
    plt.axis('equal')
    
def plot_stress(NL,U) :
    plt.figure()
    x_scatter = []
    y_scatter = []
    color = []
    for i in range(len(NL)-1) :
        x_scatter.append(np.linspace(NL[i,0],NL[i+1,0],100))
        y_scatter.append(np.linspace(NL[i,1],NL[i+1,1],100))
        color.append(np.linspace(sigma[i],sigma[i+1],100))
    #norm_x = Normalize(np.abs(color.min()),np.abs(color.max()))
    cmap = plt.get_cmap('jet')
    plt.scatter(x_scatter,y_scatter,c = color,cmap = cmap,s=10, edgecolor = 'none' )
    plt.colorbar(label='Contrainte (en MPa)') #ScalarMappable(norm = norm_x, cmap = cmap ))
    return

def plot_deformee(NL,U,scale) :
    plt.scatter(NL[:,0],[0 for x in NL])
    plt.scatter(NL[:,0]+scale*U[:].flatten(),[0 for x in NL])
    return